In [1]:
import os; os.chdir('/Users/sqian/MKTSRV')
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import datetime
import matplotlib as mpl
import re
import xlrd
from datetime import date
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from math import log10, floor
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, littleQA, totalUP
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame
from src.utility.GeneralUtility import date_range, random_string, timer

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 1A) SET day range based on Alert Volume

In [2]:
# rangeofdays = 7
# book = 'BTCUSD'

# alerts = pd.read_excel('~/Documents/vmfldr/Alerts-AllRows_gemini_20190301-20190829.xlsx')

# alertsspoof = alerts.loc[alerts['Alert Code'].isin([4022,4023,4032])]
# alertsspoof_day = alertsspoof.groupby('Date').count().rolling('{}d'.format(rangeofdays)).sum()
# firstdate = alertsspoof_day.loc[alertsspoof_day['Alert Code'] == alertsspoof_day['Alert Code'].max()].index[0] - datetime.timedelta(days=rangeofdays)

#### 1B) SET day range manually

In [3]:
firstdate = datetime.date(2019,4,1)
rangeofdays = 30
booklist = ['ZECETH','BTCUSD']

#### 2) Date List Creation

In [4]:
datelist = pd.date_range(firstdate, periods=rangeofdays).astype(str).str.replace('-','').tolist()
print('min: {}'.format(min(datelist)),'max: {}'.format(max(datelist)), sep='\n')

min: 20190401
max: 20190430


#### 3) GET that range of dates for BTCUSD

In [5]:
breakpoint = 10000000
fulldf = pd.DataFrame()
fulldfdict = {}

for j in booklist:
    appendeddf = pd.DataFrame()
    for i in datelist:
        temp = pd.read_csv('~/Downloads/{}.orders_{}.csv'.format(i,j))
        appendeddf = appendeddf.append(temp)
        print(j, i, temp.shape, appendeddf.shape)
    fulldf = fulldf.append(appendeddf, sort=False)
    fulldfdict[j] = appendeddf
    
    for i in range(0,math.ceil(appendeddf.shape[0]/breakpoint)):
        startpoint = i * breakpoint
        endpoint = (i+1) * breakpoint
        appendeddf.iloc[startpoint:endpoint].to_pickle('~/Desktop/raw_april{}_part{}.pkl'.format(j,i+1))

fulldfdict.keys()

ZECETH 20190401 (166443, 23) (166443, 23)
ZECETH 20190402 (302048, 23) (468491, 23)
ZECETH 20190403 (232796, 23) (701287, 23)
ZECETH 20190404 (159318, 23) (860605, 23)
ZECETH 20190405 (161275, 23) (1021880, 23)
ZECETH 20190406 (110592, 23) (1132472, 23)
ZECETH 20190407 (136966, 23) (1269438, 23)
ZECETH 20190408 (154306, 23) (1423744, 23)
ZECETH 20190409 (110818, 23) (1534562, 23)
ZECETH 20190410 (102114, 23) (1636676, 23)
ZECETH 20190411 (146469, 23) (1783145, 23)
ZECETH 20190412 (98083, 23) (1881228, 23)
ZECETH 20190413 (82313, 23) (1963541, 23)
ZECETH 20190414 (75486, 23) (2039027, 23)
ZECETH 20190415 (94996, 23) (2134023, 23)
ZECETH 20190416 (87523, 23) (2221546, 23)
ZECETH 20190417 (106082, 23) (2327628, 23)
ZECETH 20190418 (112922, 23) (2440550, 23)
ZECETH 20190419 (63084, 23) (2503634, 23)
ZECETH 20190420 (52976, 23) (2556610, 23)
ZECETH 20190421 (90297, 23) (2646907, 23)
ZECETH 20190422 (109671, 23) (2756578, 23)
ZECETH 20190423 (117570, 23) (2874148, 23)
ZECETH 20190424 (106691

dict_keys(['ZECETH', 'BTCUSD'])

#### 4) GENERATE random strings to replace account_Id

In [10]:
accIDmap = pd.DataFrame(pd.unique(fulldf['Account ID'])).rename(columns={0:'acc'})

accIDmap['accSTR'] = accIDmap['acc'].astype(str)
accIDmap['accountANON'] = accIDmap['acc'].apply(lambda x: random_string(12))
anondictpart = accIDmap.drop(columns='acc').set_index('accSTR')
anondict = anondictpart.to_dict()['accountANON']

In [12]:
anondictpart.to_csv('~/Desktop/accID-anonstring-map-CONFIDENTIAL_FOR_INTERNAL_ONLY')

#### 5) ANONYMIZE data & export

In [7]:
for j in booklist:
    anonymizeddf = pd.DataFrame(fulldfdict[j])
    anonymizeddf['Account ID'] = anonymizeddf['Account ID'].astype(str).map(anondict)
    fulldfdict['anon_{}'.format(j)] = anonymizeddf
    print(j,anonymizeddf.shape)
    
    dftemp = fulldfdict['anon_{}'.format(j)]
    for i in range(0,math.ceil(dftemp.shape[0]/breakpoint)):
        startpoint = i * breakpoint
        endpoint = (i+1) * breakpoint
        dftemp.iloc[startpoint:endpoint].to_pickle('~/Desktop/anon_april{}_part{}.pkl'.format(j,i+1))

ZECETH (3495658, 23)
BTCUSD (33454826, 23)


#### 5) check 100% mapping (any numbers <12 indicate a problem)

##### (no account numbers are 12 characters long, and this is the number of characters in the anonymized account string)

In [8]:
for j in booklist:
    print(j, fulldfdict['anon_{}'.format(j)]['Account ID'].apply(len).min())

ZECETH 12
BTCUSD 12
